In [31]:
import dicom
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import math
% matplotlib inline

data_dir = os.getcwd() + '\\sample_images\\'
patients = os.listdir(data_dir)
labels_df = pd.read_csv('stage1_labels.csv',index_col = 0)

print(data_dir)
print(patients)
labels_df.head()

C:\Users\azkei\Desktop\lung_cancer_data\sample_images\
['00cba091fa4ad62cc3200a657aeb957e', '0a099f2549429d29b32f349e95fb2244', '0a0c32c9e08cc2ea76a71649de56be6d', '0a38e7597ca26f9374f8ea2770ba870d', '0acbebb8d463b4b9ca88cf38431aac69', '0b20184e0cd497028bdd155d9fb42dc9', '0bd0e3056cbf23a1cb7f0f0b18446068', '0c0de3749d4fe175b7a5098b060982a1', '0c37613214faddf8701ca41e6d43f56e', '0c59313f52304e25d5a7dcf9877633b1', '0c60f4b87afcb3e2dfa65abbbf3ef2f9', '0c98fcb55e3f36d0c2b6507f62f4c5f1', '0c9d8314f9c69840e25febabb1229fa4', '0ca943d821204ceb089510f836a367fd', '0d06d764d3c07572074d468b4cff954f', '0d19f1c627df49eb223771c28548350e', '0d2fcf787026fece4e57be167d079383', '0d941a3ad6c889ac451caf89c46cb92a', '0ddeb08e9c97227853422bd71a2a695e', '0de72529c30fe642bc60dcb75c87f6bd']


,cancer
id,
0015ceb851d7251b8f399e39779d1e7d,1
0030a160d58723ff36d73f41b170ec21,0
003f41c78e6acfa92430a057ac0b306e,0
006b96310a37b36cccb2ab48d10b49a3,1
008464bb8521d09a42985dd8add3d0d2,1


In [19]:
# Iterate through 5 patients in sample_image - checking out attributes
for patient in patients[:5]:
    label = labels_df.get_value(patient,'cancer')
    path = data_dir + patient
    # using dicom to read the dicom files using full patient path
    slices = [dicom.read_file(path+ '/' + s) for s in os.listdir(path)]
    # sorting dicom files using lambda function
    # x - referred to dicom files
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
    print(len(slices), slices[0].pixel_array.shape)

134 (512, 512)
128 (512, 512)
133 (512, 512)
110 (512, 512)
203 (512, 512)


<p>IMPORTANT NOTE: Not all images are the same size.......depth isnt the same and width, height are too large for CNN </p>

In [21]:
# No. of Instances in the sample_image
len(patients)

20

In [50]:
# Pre processing data
IMG_PX_SIZE = 50
HM_SLICES = 20

def chunks(l, n):
    # Credit: Ned Batchelder
    # Link: http://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]
        

def mean(l):
    return sum(l)/len(l)

def process_data(patient, labels_df, img_px_size = 50, hm_slices=20, visualize = False):
    try:
        label = labels_df.get_value(patient, 'cancer')
        path = data_dir + patient
        slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
        slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
        new_slices = []

        slices = [cv2.resize(np.array(each_slice.pixel_array),(IMG_PX_SIZE,IMG_PX_SIZE)) for each_slice in slices]

        chunk_sizes = math.ceil(len(slices) / HM_SLICES)

        for slice_chunk in chunks(slices, chunk_sizes):
            slice_chunk = list(map(mean, zip(*slice_chunk)))
            new_slices.append(slice_chunk)

        if len(new_slices) == HM_SLICES-1:
            new_slices.append(new_slices[-1])

        if len(new_slices) == HM_SLICES-2:
            new_slices.append(new_slices[-1])
            new_slices.append(new_slices[-1])

        if len(new_slices) == HM_SLICES+2:
            new_val = list(map(mean, zip(*[new_slices[HM_SLICES-1],new_slices[HM_SLICES],])))
            del new_slices[HM_SLICES]
            new_slices[HM_SLICES-1] = new_val

        if len(new_slices) == HM_SLICES+1:
            new_val = list(map(mean, zip(*[new_slices[HM_SLICES-1],new_slices[HM_SLICES],])))
            del new_slices[HM_SLICES]
            new_slices[HM_SLICES-1] = new_val
        print(len(slices), len(new_slices))
        
        if visualize:
            fig = plt.figure()
            for num, each_slice in enumerate(slices[:12]):
                y = fig.add_subplot(3,4,num+1)
                y.imshow(each_slice)
            plt.show()
            
        if label == 1:
            label = np.array([0,1])
        elif label == 1:
            label = np.array([0,1])
        
        return np.array(new_slices),label
    
    except Exception as e:
        # again, some patients are not labeled, but JIC we still want the error if something
        # else is wrong with our code
        print(str(e))
        
much_data = []
for num, patient in enumerate(patients[:5]):
    if num%100 == 0:
        print(num)
    try:
        img_data, label = process_data(patient,labels_df,img_px_size=IMG_PX_SIZE, hm_slices=HM_SLICES)
        much_data.append([img_data, label])
    except KeyError as e:
        print("This is unlabeled data")
    
np.save('muchdata--{}--{}--{}.npy'.format(IMG_PX_SIZE,IMG_PX_SIZE,HM_SLICES),much_data)

0
134 20
128 20
133 20
110 20
203 20


<p> Each of this instance is a patient </p>